In [1]:
from astropy.io import fits
from astropy.table import Table
import numpy as np
import matplotlib.pylab as plt 
import matplotlib.lines as mlines
from matplotlib.legend import Legend
from pythonds.basic.stack import Stack
from math import *
from sklearn.neighbors import KDTree
from lrg_plot_functions import *
from lrg_sum_functions import *
from cosmo_Calc import *
from divideByTwo import *
# from read_data import *
from nearNeighbors import *
from scipy import stats
from astropy import stats
import healpy as hp
from localBKG_and_interlopersHEALPix import *
# import healpy as hpy
from makeCutoutTableFunc import *
from plotHistogramProjections3D import *
from read_bricks import *
from mpl_toolkits.axes_grid1 import make_axes_locatable
import random

In [20]:
def read_data_new(SpecObj_data_south, SDSS_data, DECaLS_data):
    # Read in data from SDSS file

    # Redshift of galaxies according to sdss
    z = []
    z = SDSS_data.field('Z')

    # Unique ID for sources in SDSS
    specobjid = []
    specobjid = SDSS_data.field('SPECOBJID')

    # Class of object
    gal_class = []
    gal_class = SDSS_data.field('CLASS')

    # What survey the data is from
    survey = []
    survey = SDSS_data.field('SURVEY')

    # SPECPRIMARY; set to 1 for primary observation of object, 0 otherwise
    spec = []
    spec = SDSS_data.field('SPECPRIMARY')

    # Bitmask of spectroscopic warning values; need set to 0
    zwarn_noqso = []
    zwarn_noqso = SDSS_data.field('ZWARNING_NOQSO')

    # Spectroscopic classification for certain redshift?
    class_noqso = []
    class_noqso = SDSS_data.field('CLASS_NOQSO')

    # Array for LRG targets
    targets = []
    targets = SDSS_data.field('BOSS_TARGET1')

    # Array for tile number (cut for LOWZ LRGs is tile >= 10324)
    tile = []
    tile = SDSS_data.field('TILE')

    # Array for chunk number (cut for CMASS is chunk != 'boss1' or 'boss2'
    chunk = []
    chunk = SDSS_data.field('CHUNK')

    plate = []
    plate = SDSS_data.field('PLATEID')

    fiberid = []
    fiberid = SDSS_data.field('FIBERID')


    print(len(z))
    print('done reading in SDSS')
    
    
    # Read in data from SDSS row matched DECaLS file

    # release 
    release_MATCHED = []
    release_MATCHED = SpecObj_data.field('release')

    # Object ID from survey file; value -1 for non-matches
    objid_MATCHED = []
    objid_MATCHED = SpecObj_data.field('OBJID')

    # Add brickid
    brickid_MATCHED = []
    brickid_MATCHED = SpecObj_data.field('BRICKID')

    # Add brickname
    brickname_MATCHED = []
    brickname_MATCHED = SpecObj_data.field('BRICKNAME')

    # Only galaxies included
    gal_type_MATCHED = []
    gal_type_MATCHED = SpecObj_data.field('TYPE')

    # RA
    ra_MATCHED = []
    ra_MATCHED = SpecObj_data.field('RA')

    # Dec
    dec_MATCHED = []
    dec_MATCHED = SpecObj_data.field('DEC')

    # flux_g
    gflux_MATCHED = []
    gflux_MATCHED = SpecObj_data.field('FLUX_G')

    # flux_r
    rflux_MATCHED = []
    rflux_MATCHED = SpecObj_data.field('FLUX_R')

    # flux_z
    zflux_MATCHED = []
    zflux_MATCHED = SpecObj_data.field('FLUX_Z')

    # flux_W1
    w1flux_MATCHED = []
    w1flux_MATCHED = SpecObj_data.field('flux_w1')

    # flux_W2
    w2flux_MATCHED = []
    w2flux_MATCHED = SpecObj_data.field('flux_w2')

    # flux_W3
    w3flux_MATCHED = []
    w3flux_MATCHED = SpecObj_data.field('flux_w3')

    # flux_W4
    w4flux_MATCHED = []
    w4flux_MATCHED = SpecObj_data.field('flux_w4')

    # nobs == number of images that contribute to the central pixel
    # nobs_g
    gobs_MATCHED = []
    gobs_MATCHED = SpecObj_data.field('NOBS_G')

    # nobs_r
    robs_MATCHED = []
    robs_MATCHED = SpecObj_data.field('NOBS_R')

    # nobs_z
    zobs_MATCHED = []
    zobs_MATCHED = SpecObj_data.field('NOBS_Z')

    #flux errors
    gflux_ivar_MATCHED = []
    gflux_ivar_MATCHED = SpecObj_data.field('flux_ivar_g')

    rflux_ivar_MATCHED = []
    rflux_ivar_MATCHED = SpecObj_data.field('flux_ivar_r')

    zflux_ivar_MATCHED = []
    zflux_ivar_MATCHED = SpecObj_data.field('flux_ivar_z')

    w1flux_ivar_MATCHED = []
    w1flux_ivar_MATCHED = SpecObj_data.field('flux_ivar_w1')

    w2flux_ivar_MATCHED = []
    w2flux_ivar_MATCHED = SpecObj_data.field('flux_ivar_w2')

    w3flux_ivar_MATCHED = []
    w3flux_ivar_MATCHED = SpecObj_data.field('flux_ivar_w3')

    w4flux_ivar_MATCHED = []
    w4flux_ivar_MATCHED = SpecObj_data.field('flux_ivar_w4')

    mw_transmission_g_MATCHED = []
    mw_transmission_g_MATCHED = SpecObj_data.field('mw_transmission_g')

    mw_transmission_r_MATCHED = []
    mw_transmission_r_MATCHED = SpecObj_data.field('mw_transmission_r')

    mw_transmission_z_MATCHED = []
    mw_transmission_z_MATCHED = SpecObj_data.field('mw_transmission_z')

    mw_transmission_w1_MATCHED = []
    mw_transmission_w1_MATCHED = SpecObj_data.field('mw_transmission_w1')

    mw_transmission_w2_MATCHED = []
    mw_transmission_w2_MATCHED = SpecObj_data.field('mw_transmission_w2')

    mw_transmission_w3_MATCHED = []
    mw_transmission_w3_MATCHED = SpecObj_data.field('mw_transmission_w3')

    mw_transmission_w4_MATCHED = []
    mw_transmission_w4_MATCHED = SpecObj_data.field('mw_transmission_w4')

    # maskbits
    maskbits_MATCHED = []
    maskbits_MATCHED = SpecObj_data.field('maskbits')


#     print(len(objid_MATCHED[np.where(objid_MATCHED != -1)]))
    print(len(objid_MATCHED))
    print('done reading in DECaLS-SDSS matched file')
    
    
    # Read in data from DECaLS bricks

    # release 
    release_ALL = []
    release_ALL = DECaLS_data.field('release')

    # Object ID from survey file
    objid_ALL = []
    objid_ALL = DECaLS_data.field('OBJID')
    # print(len(objid_ALL))

    # Add brickid
    brickid_ALL = []
    brickid_ALL = DECaLS_data.field('BRICKID')

    # Add brickname
    brickname_ALL = []
    brickname_ALL = DECaLS_data.field('BRICKNAME')

    # Only galaxies included
    gal_type_ALL = []
    gal_type_ALL = DECaLS_data.field('TYPE')

    # RA
    ra_ALL = []
    ra_ALL = DECaLS_data.field('RA')

    # Dec
    dec_ALL = []
    dec_ALL = DECaLS_data.field('DEC')

    # flux_g
    gflux_ALL = []
    gflux_ALL = DECaLS_data.field('FLUX_G')

    # flux_r
    rflux_ALL = []
    rflux_ALL = DECaLS_data.field('FLUX_R')

    # flux_z
    zflux_ALL = []
    zflux_ALL = DECaLS_data.field('FLUX_Z')

    # flux_W1
    w1flux_ALL = []
    w1flux_ALL = DECaLS_data.field('flux_w1')

    # flux_W2
    w2flux_ALL = []
    w2flux_ALL = DECaLS_data.field('flux_w2')

    # flux_W3
    w3flux_ALL = []
    w3flux_ALL = DECaLS_data.field('flux_w3')

    # flux_W4
    w4flux_ALL = []
    w4flux_ALL = DECaLS_data.field('flux_w4')

    # nobs == number of images that contribute to the central pixel
    # nobs_g
    gobs_ALL = []
    gobs_ALL = DECaLS_data.field('NOBS_G')

    # nobs_r
    robs_ALL = []
    robs_ALL = DECaLS_data.field('NOBS_R')

    # nobs_z
    zobs_ALL = []
    zobs_ALL = DECaLS_data.field('NOBS_Z')

    # flux errors
    gflux_ivar_ALL = []
    gflux_ivar_ALL = DECaLS_data.field('flux_ivar_g')

    rflux_ivar_ALL = []
    rflux_ivar_ALL = DECaLS_data.field('flux_ivar_r')

    zflux_ivar_ALL = []
    zflux_ivar_ALL = DECaLS_data.field('flux_ivar_z')

    w1flux_ivar_ALL = []
    w1flux_ivar_ALL = DECaLS_data.field('flux_ivar_w1')

    w2flux_ivar_ALL = []
    w2flux_ivar_ALL = DECaLS_data.field('flux_ivar_w2')

    w3flux_ivar_ALL = []
    w3flux_ivar_ALL = DECaLS_data.field('flux_ivar_w3')

    w4flux_ivar_ALL = []
    w4flux_ivar_ALL = DECaLS_data.field('flux_ivar_w4')

    mw_transmission_g_ALL = []
    mw_transmission_g_ALL = DECaLS_data.field('mw_transmission_g')

    mw_transmission_r_ALL = []
    mw_transmission_r_ALL = DECaLS_data.field('mw_transmission_r')

    mw_transmission_z_ALL = []
    mw_transmission_z_ALL = DECaLS_data.field('mw_transmission_z')

    mw_transmission_w1_ALL = []
    mw_transmission_w1_ALL = DECaLS_data.field('mw_transmission_w1')

    mw_transmission_w2_ALL = []
    mw_transmission_w2_ALL = DECaLS_data.field('mw_transmission_w2')

    mw_transmission_w3_ALL = []
    mw_transmission_w3_ALL = DECaLS_data.field('mw_transmission_w3')

    mw_transmission_w4_ALL = []
    mw_transmission_w4_ALL = DECaLS_data.field('mw_transmission_w4')

    # maskbits
    maskbits_ALL = []
    maskbits_ALL = DECaLS_data.field('maskbits')


    print(len(objid_ALL))
    print('done reading in DECaLS bricks')
    
    
    # extract LRGs from MATCHED file
    
    # Create a unique identifier by combinding BRICKID and OBJID for row-matched file

    id_MATCHED = []

    for i in range(len(objid_MATCHED)):
        if (objid_MATCHED[i] == -1):
            id_MATCHED.append(-1)
        else:
            temp1 = str(release_MATCHED[i]) + str(brickid_MATCHED[i]) + str(objid_MATCHED[i])
            id_MATCHED.append(temp1)

    id_MATCHED = [int(i) for i in id_MATCHED]
    id_MATCHED = np.array(id_MATCHED)


    print(len(id_MATCHED))
    print('done creating unique IDs for matched file')
    
    
    # LRG-only criteria

    LRG_only = ((((targets & 2**0)!=0) | ((targets & 2**1)!=0)) & (gobs_MATCHED >= 2.) & (robs_MATCHED >= 2.) & (zobs_MATCHED >= 2.) & (gflux_MATCHED > 0.) & (rflux_MATCHED > 0.) & (zflux_MATCHED > 0.) & (objid_MATCHED != -1) & ((gal_type_MATCHED == 'SER') | (gal_type_MATCHED == "DEV") | (gal_type_MATCHED == "EXP") | (gal_type_MATCHED == "REX")) & (tile >= 10324) & (gal_class == 'GALAXY') & (spec == 1) & (zwarn_noqso == 0) & (class_noqso == 'GALAXY') & ((survey == 'sdss') | (survey == 'boss')))

    z_LRG = z[LRG_only]
    ra_LRG = ra_MATCHED[LRG_only]
    dec_LRG = dec_MATCHED[LRG_only]
    gflux_LRG = gflux_MATCHED[LRG_only]
    rflux_LRG = rflux_MATCHED[LRG_only]
    zflux_LRG = zflux_MATCHED[LRG_only]
    w1flux_LRG = w1flux_MATCHED[LRG_only]
    w2flux_LRG = w2flux_MATCHED[LRG_only]
    w3flux_LRG = w3flux_MATCHED[LRG_only]
    w4flux_LRG = w4flux_MATCHED[LRG_only]
    gflux_ivar_LRG = gflux_ivar_MATCHED[LRG_only]
    rflux_ivar_LRG = rflux_ivar_MATCHED[LRG_only]
    zflux_ivar_LRG = zflux_ivar_MATCHED[LRG_only]
    w1flux_ivar_LRG = w1flux_ivar_MATCHED[LRG_only]
    w2flux_ivar_LRG = w2flux_ivar_MATCHED[LRG_only]
    w3flux_ivar_LRG = w3flux_ivar_MATCHED[LRG_only]
    w4flux_ivar_LRG = w4flux_ivar_MATCHED[LRG_only]
    id_LRG = id_MATCHED[LRG_only]
    specobjid_LRG = specobjid[LRG_only]
    mw_transmission_g_LRG = mw_transmission_g_MATCHED[LRG_only]
    mw_transmission_r_LRG = mw_transmission_r_MATCHED[LRG_only]
    mw_transmission_z_LRG = mw_transmission_z_MATCHED[LRG_only]
    mw_transmission_w1_LRG = mw_transmission_w1_MATCHED[LRG_only]
    mw_transmission_w2_LRG = mw_transmission_w2_MATCHED[LRG_only]
    mw_transmission_w3_LRG = mw_transmission_w3_MATCHED[LRG_only]
    mw_transmission_w4_LRG = mw_transmission_w4_MATCHED[LRG_only]
    maskbits_LRG = maskbits_MATCHED[LRG_only]
    
    
    # remove LRGs from ALL file
    
    # Create a unique identifier by combinding BRICKID and OBJID for DECaLS brick

    id_ALL = []

    for i in range(len(objid_ALL)):
        temp2 = str(release_ALL[i]) + str(brickid_ALL[i]) + str(objid_ALL[i])
        id_ALL.append(temp2)

    id_ALL = [int(i) for i in id_ALL]
    id_ALL = np.array(id_ALL)

    print('length of id_ALL: ', len(id_ALL))
    print('done creating unique IDs for brick file')
    
    
    # find all instances of LRGs in ALL file
    
    id_cut = id_ALL
    
    array, ind_ALL, ind_LRG = np.intersect1d(id_ALL, id_LRG, return_indices=True)
    print(len(ind_ALL)) # ind_ALL gives the indices of id_ALL where id_ALL have common values with id_LRG
    id_cut[ind_ALL] = 0

    print(len(id_cut))
    print(len(id_cut[np.where(id_ALL == 0)]))
    
    
    survey_cut = ((id_cut != 0) & (gobs_ALL >= 2.) & (robs_ALL >= 2.) & (zobs_ALL >= 2.) & (gflux_ALL > 0.) & (rflux_ALL > 0.) & (zflux_ALL > 0.) & ((gal_type_ALL == 'SER') | (gal_type_ALL == "DEV") | (gal_type_ALL == "EXP") | (gal_type_ALL == "REX")))


    ra_BKG = ra_ALL[survey_cut]
    dec_BKG = dec_ALL[survey_cut]
    gflux_BKG = gflux_ALL[survey_cut]
    rflux_BKG = rflux_ALL[survey_cut]
    zflux_BKG = zflux_ALL[survey_cut]
    w1flux_BKG = w1flux_ALL[survey_cut]
    w2flux_BKG = w2flux_ALL[survey_cut]
    w3flux_BKG = w3flux_ALL[survey_cut]
    w4flux_BKG = w4flux_ALL[survey_cut]
    gflux_ivar_BKG = gflux_ivar_ALL[survey_cut]
    rflux_ivar_BKG = rflux_ivar_ALL[survey_cut]
    zflux_ivar_BKG = zflux_ivar_ALL[survey_cut]
    w1flux_ivar_BKG = w1flux_ivar_ALL[survey_cut]
    w2flux_ivar_BKG = w2flux_ivar_ALL[survey_cut]
    w3flux_ivar_BKG = w3flux_ivar_ALL[survey_cut]
    w4flux_ivar_BKG = w4flux_ivar_ALL[survey_cut]
    mw_transmission_g_BKG = mw_transmission_g_ALL[survey_cut]
    mw_transmission_r_BKG = mw_transmission_r_ALL[survey_cut]
    mw_transmission_z_BKG = mw_transmission_z_ALL[survey_cut]
    mw_transmission_w1_BKG = mw_transmission_w1_ALL[survey_cut]
    mw_transmission_w2_BKG = mw_transmission_w2_ALL[survey_cut]
    mw_transmission_w3_BKG = mw_transmission_w3_ALL[survey_cut]
    mw_transmission_w4_BKG = mw_transmission_w4_ALL[survey_cut]
    maskbits_BKG = maskbits_ALL[survey_cut]
    
    return maskbits_LRG, maskbits_BKG, id_LRG, ra_LRG, dec_LRG, ra_BKG, dec_BKG, z_LRG, gflux_LRG, rflux_LRG, zflux_LRG, w1flux_LRG, w2flux_LRG, w3flux_LRG, w4flux_LRG, gflux_BKG, rflux_BKG, zflux_BKG, w1flux_BKG, w2flux_BKG, w3flux_BKG, w4flux_BKG, gflux_ivar_LRG, rflux_ivar_LRG, zflux_ivar_LRG, w1flux_ivar_LRG, w2flux_ivar_LRG, w3flux_ivar_LRG, w4flux_ivar_LRG, gflux_ivar_BKG, rflux_ivar_BKG, zflux_ivar_BKG, w1flux_ivar_BKG, w2flux_ivar_BKG, w3flux_ivar_BKG, w4flux_ivar_BKG, mw_transmission_g_LRG, mw_transmission_r_LRG, mw_transmission_z_LRG, mw_transmission_g_BKG, mw_transmission_r_BKG, mw_transmission_z_BKG, mw_transmission_w1_LRG, mw_transmission_w2_LRG, mw_transmission_w3_LRG, mw_transmission_w4_LRG, mw_transmission_w1_BKG, mw_transmission_w2_BKG, mw_transmission_w3_BKG, mw_transmission_w4_BKG
    



In [21]:
# read in data from fits files

hdu_north = fits.open('/Users/mtownsend/anaconda/Data/survey-dr9-north-specObj-dr16.fits') # this matches SDSS LRGs to MzLS/BASS;                                                               # ONLY GIVES SOURCES THAT ARE IN SDSS and LEGACY SURVEYS
hdu_south = fits.open('/Users/mtownsend/anaconda/Data/survey-dr9-south-specObj-dr16.fits') # this matches SDSS LRGs to DECaLS;
                                                                 # ONLY GIVES SOURCES THAT ARE IN SDSS and LEGACY SURVEYS
hdu_sdss = fits.open('/Users/mtownsend/anaconda/Data/specObj-dr16.fits') # this is SDSS redshifts etc for LRGs



In [46]:
# # read in data from fits files

# hdu_north = fits.open('/Users/mtownsend/anaconda/Data/survey-dr9-north-specObj-dr16.fits') # this matches SDSS LRGs to MzLS/BASS;                                                               # ONLY GIVES SOURCES THAT ARE IN SDSS and LEGACY SURVEYS
# hdu_south = fits.open('/Users/mtownsend/anaconda/Data/survey-dr9-south-specObj-dr16.fits') # this matches SDSS LRGs to DECaLS;
#                                                                  # ONLY GIVES SOURCES THAT ARE IN SDSS and LEGACY SURVEYS
# hdu_sdss = fits.open('/Users/mtownsend/anaconda/Data/specObj-dr16.fits') # this is SDSS redshifts etc for LRGs



hdu_sweep1 = fits.open('/Users/mtownsend/documents/LRG_Data/expanded-data/sweep-100p015-110p020.fits') # this is one sweep file of the DECaLS data
hdu_sweep2 = fits.open('/Users/mtownsend/documents/LRG_Data/expanded-data/sweep-100p015-110p020.fits') # this is one sweep file of the DECaLS data
hdu_sweep3 = fits.open('/Users/mtownsend/documents/LRG_Data/expanded-data/sweep-100p025-110p030.fits') 
hdu_sweep4 = fits.open('/Users/mtownsend/documents/LRG_Data/expanded-data/sweep-100p030-110p035.fits')
hdu_sweep5 = fits.open('/Users/mtownsend/documents/LRG_Data/expanded-data/sweep-100p035-110p040.fits')
hdu_sweep6 = fits.open('/Users/mtownsend/documents/LRG_Data/expanded-data/sweep-110m005-120p000.fits')
hdu_sweep7 = fits.open('/Users/mtownsend/documents/LRG_Data/expanded-data/sweep-110p005-120p010.fits')

hdu_sweep8 = fits.open('/Users/mtownsend/documents/LRG_Data/expanded-data/sweep-110p010-120p015.fits')
hdu_sweep9 = fits.open('/Users/mtownsend/documents/LRG_Data/expanded-data/sweep-110p015-120p020.fits')
hdu_sweep10 = fits.open('/Users/mtownsend/documents/LRG_Data/expanded-data/sweep-110p020-120p025.fits')

hdu_sweep11 = fits.open('/Users/mtownsend/documents/LRG_Data/expanded-data/sweep-110p030-120p035.fits') 
hdu_sweep12 = fits.open('/Users/mtownsend/documents/LRG_Data/expanded-data/sweep-120m005-130p000.fits')
hdu_sweep13 = fits.open('/Users/mtownsend/documents/LRG_Data/expanded-data/sweep-120m010-130m005.fits') 
hdu_sweep14 = fits.open('/Users/mtownsend/documents/LRG_Data/expanded-data/sweep-120p000-130p005.fits')
hdu_sweep15 = fits.open('/Users/mtownsend/documents/LRG_Data/expanded-data/sweep-120p005-130p010.fits')
hdu_sweep16 = fits.open('/Users/mtownsend/documents/LRG_Data/expanded-data/sweep-120p010-130p015.fits')

hdu_sweep17 = fits.open('/Users/mtownsend/documents/LRG_Data/expanded-data/sweep-120p015-130p020.fits') 
hdu_sweep18 = fits.open('/Users/mtownsend/documents/LRG_Data/expanded-data/sweep-120p020-130p025.fits') 
hdu_sweep19 = fits.open('/Users/mtownsend/documents/LRG_Data/expanded-data/sweep-120p025-130p030.fits')
hdu_sweep20 = fits.open('/Users/mtownsend/documents/LRG_Data/expanded-data/sweep-120p030-130p035.fits') 
hdu_sweep21 = fits.open('/Users/mtownsend/documents/LRG_Data/expanded-data/sweep-120p035-130p040.fits')

# hdu_sweep22 = fits.open('/Users/mtownsend/documents/LRG_Data/expanded-data/sweep-040p020-050p025.fits')

# hdu_sweep23 = fits.open('/Users/mtownsend/documents/LRG_Data/expanded-data/sweep-040p025-050p030.fits') 
# hdu_sweep24 = fits.open('/Users/mtownsend/documents/LRG_Data/expanded-data/sweep-050m005-060p000.fits') 
# hdu_sweep25 = fits.open('/Users/mtownsend/documents/LRG_Data/expanded-data/sweep-040p030-050p035.fits')


# hdu_sweep26 = fits.open('/Users/mtownsend/documents/LRG_Data/expanded-data/sweep-270p025-280p030.fits') 
# hdu_sweep27 = fits.open('/Users/mtownsend/documents/LRG_Data/expanded-data/sweep-270p030-280p035.fits') 

# hdu_sweep28 = fits.open('/Users/mtownsend/documents/LRG_Data/expanded-data/sweep-240m005-250p000.fits')

In [47]:
SpecObj_data_north = hdu_north[1].data
print('done')
SpecObj_data = hdu_south[1].data
print('done')
SDSS_data = hdu_sdss[1].data
print('done')
DECaLS_data1 = hdu_sweep1[1].data
print('done')
DECaLS_data2 = hdu_sweep2[1].data
print('done')
DECaLS_data3 = hdu_sweep3[1].data
print('done')
DECaLS_data4 = hdu_sweep4[1].data
print('done')
DECaLS_data5 = hdu_sweep5[1].data
print('done')
DECaLS_data6 = hdu_sweep6[1].data
print('done')
DECaLS_data7 = hdu_sweep7[1].data
print('done')
DECaLS_data8 = hdu_sweep8[1].data
print('done')
DECaLS_data9 = hdu_sweep9[1].data
print('done')
DECaLS_data10 = hdu_sweep10[1].data
print('done')
DECaLS_data11 = hdu_sweep11[1].data
print('done')
DECaLS_data12 = hdu_sweep12[1].data
print('done')
DECaLS_data13 = hdu_sweep13[1].data
print('done')
DECaLS_data14 = hdu_sweep14[1].data
print('done')
DECaLS_data15 = hdu_sweep15[1].data
print('done')
DECaLS_data16 = hdu_sweep16[1].data
print('done')
DECaLS_data17 = hdu_sweep17[1].data
print('done')
DECaLS_data18 = hdu_sweep18[1].data
print('done')
DECaLS_data19 = hdu_sweep19[1].data
print('done')
DECaLS_data20 = hdu_sweep20[1].data
print('done')
DECaLS_data21 = hdu_sweep21[1].data
print('done')
# DECaLS_data22 = hdu_sweep22[1].data
# print('done')
# DECaLS_data23 = hdu_sweep23[1].data
# print('done')
# DECaLS_data24 = hdu_sweep24[1].data
# print('done')
# DECaLS_data25 = hdu_sweep25[1].data
# print('done')
# DECaLS_data26 = hdu_sweep26[1].data
# print('done')
# DECaLS_data27 = hdu_sweep27[1].data
# print('done')
# DECaLS_data28 = hdu_sweep28[1].data
# print('done')
# DECaLS_data29 = hdu_sweep29[1].data
# print('done')
# DECaLS_data30 = hdu_sweep30[1].data
# print('done')
# DECaLS_data31 = hdu_sweep31[1].data
# print('done')
# DECaLS_data32 = hdu_sweep32[1].data
# print('done')
# DECaLS_data33 = hdu_sweep33[1].data
# print('done')
# DECaLS_data34 = hdu_sweep34[1].data
# print('done')
# DECaLS_data35 = hdu_sweep35[1].data
# print('done')
# DECaLS_data36 = hdu_sweep36[1].data
# print('done')
# DECaLS_data37 = hdu_sweep37[1].data
# print('done')
# DECaLS_data38 = hdu_sweep38[1].data
# print('done')
# DECaLS_data39 = hdu_sweep39[1].data
# print('done')
# DECaLS_data40 = hdu_sweep40[1].data
# print('done')
# DECaLS_data41 = hdu_sweep41[1].data
# print('done')
# DECaLS_data42 = hdu_sweep42[1].data
# print('done')
# DECaLS_data43 = hdu_sweep43[1].data
# print('done')



done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done


In [48]:
DECaLS_data = np.array([DECaLS_data1,
DECaLS_data2,
DECaLS_data3, 
DECaLS_data4, 
DECaLS_data5, 
DECaLS_data6, 
DECaLS_data7, 
DECaLS_data8,
DECaLS_data9, 
DECaLS_data10,
DECaLS_data11, 
DECaLS_data12, 
DECaLS_data13, 
DECaLS_data14, 
DECaLS_data15, 
DECaLS_data16, 
DECaLS_data17, 
DECaLS_data18, 
DECaLS_data19, 
DECaLS_data20]) #, 
# DECaLS_data21, 
# DECaLS_data22, 
# DECaLS_data23, 
# DECaLS_data24]) 
# , 
# DECaLS_data25]) 

In [49]:
for i in range(len(DECaLS_data)):
    maskbits_LRG, maskbits_BKG, id_LRG, ra_LRG, dec_LRG, ra_BKG, dec_BKG, z_LRG, gflux_LRG, rflux_LRG, zflux_LRG, w1flux_LRG, w2flux_LRG, w3flux_LRG, w4flux_LRG, gflux_BKG, rflux_BKG, zflux_BKG, w1flux_BKG, w2flux_BKG, w3flux_BKG, w4flux_BKG, gflux_ivar_LRG, rflux_ivar_LRG, zflux_ivar_LRG, w1flux_ivar_LRG, w2flux_ivar_LRG, w3flux_ivar_LRG, w4flux_ivar_LRG, gflux_ivar_BKG, rflux_ivar_BKG, zflux_ivar_BKG, w1flux_ivar_BKG, w2flux_ivar_BKG, w3flux_ivar_BKG, w4flux_ivar_BKG, mw_transmission_g_LRG, mw_transmission_r_LRG, mw_transmission_z_LRG, mw_transmission_g_BKG, mw_transmission_r_BKG, mw_transmission_z_BKG, mw_transmission_w1_LRG, mw_transmission_w2_LRG, mw_transmission_w3_LRG, mw_transmission_w4_LRG, mw_transmission_w1_BKG, mw_transmission_w2_BKG, mw_transmission_w3_BKG, mw_transmission_w4_BKG = read_data_new(SpecObj_data, SDSS_data, DECaLS_data[i])
    
    np.savez('/Users/mtownsend/Documents/LRG_Data/expanded-data/new-cats/100-130/raLRG/ra_LRGs-100-130-dr9-{}.npz'.format(i), ra_LRG=ra_LRG)
    np.savez('/Users/mtownsend/Documents/LRG_Data/expanded-data/new-cats/100-130/decLRG/dec_LRGs-100-130-dr9-{}.npz'.format(i), dec_LRG=dec_LRG)    
    np.savez('/Users/mtownsend/Documents/LRG_Data/expanded-data/new-cats/100-130/maskbitsLRG/maskbits_LRGs-100-130-dr9-{}.npz'.format(i), maskbits_LRG=maskbits_LRG)
    np.savez('/Users/mtownsend/Documents/LRG_Data/expanded-data/new-cats/100-130/zLRG/z_LRG-100-130-dr9-{}.npz'.format(i), z_LRG=z_LRG)
    np.savez('/Users/mtownsend/Documents/LRG_Data/expanded-data/new-cats/100-130/gfluxLRG/gflux_LRG-100-130-dr9-{}.npz'.format(i), gflux_LRG=gflux_LRG)
    np.savez('/Users/mtownsend/Documents/LRG_Data/expanded-data/new-cats/100-130/rfluxLRG/rflux_LRG-100-130-dr9-{}.npz'.format(i), rflux_LRG=rflux_LRG)
    np.savez('/Users/mtownsend/Documents/LRG_Data/expanded-data/new-cats/100-130/zfluxLRG/zflux_LRG-100-130-dr9-{}.npz'.format(i), zflux_LRG=zflux_LRG) 
    np.savez('/Users/mtownsend/Documents/LRG_Data/expanded-data/new-cats/100-130/mw_trans_g_LRG/mw_transmission_g_LRGs-100-130-dr9-{}.npz'.format(i), mw_transmission_g_LRG=mw_transmission_g_LRG)
    np.savez('/Users/mtownsend/Documents/LRG_Data/expanded-data/new-cats/100-130/mw_trans_r_LRG/mw_transmission_r_LRGs-100-130-dr9-{}.npz'.format(i), mw_transmission_r_LRG=mw_transmission_r_LRG)
    np.savez('/Users/mtownsend/Documents/LRG_Data/expanded-data/new-cats/100-130/mw_trans_z_LRG/mw_transmission_z_LRGs-100-130-dr9-{}.npz'.format(i), mw_transmission_z_LRG=mw_transmission_z_LRG)
    np.savez('/Users/mtownsend/Documents/LRG_Data/expanded-data/new-cats/100-130/mw_trans_w1_LRG/mw_transmission_w1_LRGs-100-130-dr9-{}.npz'.format(i), mw_transmission_w1_LRG=mw_transmission_w1_LRG)
    np.savez('/Users/mtownsend/Documents/LRG_Data/expanded-data/new-cats/100-130/mw_trans_w2_LRG/mw_transmission_w2_LRGs-100-130-dr9-{}.npz'.format(i), mw_transmission_w2_LRG=mw_transmission_w2_LRG)
    np.savez('/Users/mtownsend/Documents/LRG_Data/expanded-data/new-cats/100-130/mw_trans_w3_LRG/mw_transmission_w3_LRGs-100-130-dr9-{}.npz'.format(i), mw_transmission_w3_LRG=mw_transmission_w3_LRG)
    np.savez('/Users/mtownsend/Documents/LRG_Data/expanded-data/new-cats/100-130/mw_trans_w4_LRG/mw_transmission_w4_LRGs-100-130-dr9-{}.npz'.format(i), mw_transmission_w4_LRG=mw_transmission_w4_LRG)
    
    np.savez('/Users/mtownsend/Documents/LRG_Data/expanded-data/new-cats/100-130/raBKG/ra_BKG-100-130-dr9-{}.npz'.format(i), ra_BKG=ra_BKG)
    np.savez('/Users/mtownsend/Documents/LRG_Data/expanded-data/new-cats/100-130/decBKG/dec_BKG-100-130-dr9-{}.npz'.format(i), dec_BKG=dec_BKG)
    np.savez('/Users/mtownsend/Documents/LRG_Data/expanded-data/new-cats/100-130/maskbitsBKG/maskbits_BKG-100-130-dr9-{}.npz'.format(i), maskbits_BKG=maskbits_BKG)
    np.savez('/Users/mtownsend/Documents/LRG_Data/expanded-data/new-cats/100-130/gfluxBKG/gflux_BKG-100-130-dr9-{}.npz'.format(i), gflux_BKG=gflux_BKG)
    np.savez('/Users/mtownsend/Documents/LRG_Data/expanded-data/new-cats/100-130/rfluxBKG/rflux_BKG-100-130-dr9-{}.npz'.format(i), rflux_BKG=rflux_BKG)
    np.savez('/Users/mtownsend/Documents/LRG_Data/expanded-data/new-cats/100-130/zfluxBKG/zflux_BKG-100-130-dr9-{}.npz'.format(i), zflux_BKG=zflux_BKG)
    np.savez('/Users/mtownsend/Documents/LRG_Data/expanded-data/new-cats/100-130/mw_trans_g_BKG/mw_transmission_g_BKG-100-130-dr9-{}.npz'.format(i), mw_transmission_g_BKG=mw_transmission_g_BKG)
    np.savez('/Users/mtownsend/Documents/LRG_Data/expanded-data/new-cats/100-130/mw_trans_r_BKG/mw_transmission_r_BKG-100-130-dr9-{}.npz'.format(i), mw_transmission_r_BKG=mw_transmission_r_BKG)
    np.savez('/Users/mtownsend/Documents/LRG_Data/expanded-data/new-cats/100-130/mw_trans_z_BKG/mw_transmission_z_BKG-100-130-dr9-{}.npz'.format(i), mw_transmission_z_BKG=mw_transmission_z_BKG)
    np.savez('/Users/mtownsend/Documents/LRG_Data/expanded-data/new-cats/100-130/mw_trans_w1_BKG/mw_transmission_w1_BKG-100-130-dr9-{}.npz'.format(i), mw_transmission_w1_BKG=mw_transmission_w1_BKG)
    np.savez('/Users/mtownsend/Documents/LRG_Data/expanded-data/new-cats/100-130/mw_trans_w2_BKG/mw_transmission_w2_BKG-100-130-dr9-{}.npz'.format(i), mw_transmission_w2_BKG=mw_transmission_w2_BKG)
    np.savez('/Users/mtownsend/Documents/LRG_Data/expanded-data/new-cats/100-130/mw_trans_w3_BKG/mw_transmission_w3_BKG-100-130-dr9-{}.npz'.format(i), mw_transmission_w3_BKG=mw_transmission_w3_BKG)
    np.savez('/Users/mtownsend/Documents/LRG_Data/expanded-data/new-cats/100-130/mw_trans_w4_BKG/mw_transmission_w4_BKG-100-130-dr9-{}.npz'.format(i), mw_transmission_w4_BKG=mw_transmission_w4_BKG)
    print('end save ', i)

print('end saving files')  

5789200
done reading in SDSS
5789200
done reading in DECaLS-SDSS matched file
204285
done reading in DECaLS bricks
5789200
done creating unique IDs for matched file
length of id_ALL:  204285
done creating unique IDs for brick file
0
204285
0
end save  0
5789200
done reading in SDSS
5789200
done reading in DECaLS-SDSS matched file
204285
done reading in DECaLS bricks
5789200
done creating unique IDs for matched file
length of id_ALL:  204285
done creating unique IDs for brick file
0
204285
0
end save  1
5789200
done reading in SDSS
5789200
done reading in DECaLS-SDSS matched file
1968754
done reading in DECaLS bricks
5789200
done creating unique IDs for matched file
length of id_ALL:  1968754
done creating unique IDs for brick file
0
1968754
0
end save  2
5789200
done reading in SDSS
5789200
done reading in DECaLS-SDSS matched file
1848923
done reading in DECaLS bricks
5789200
done creating unique IDs for matched file
length of id_ALL:  1848923
done creating unique IDs for brick file
0


In [ ]:


data_raLRG = np.load('/Users/mtownsend/Documents/LRG_Data/ra_LRGs-dr9-240-270-3.npz')
raLRG = data_raLRG['ra_LRG']
print(len(raLRG))
print(raLRG[0:10])

print()

data_decLRG = np.load('/Users/mtownsend/Documents/LRG_Data/dec_LRGs-dr9-240-270-3.npz')
decLRG = data_decLRG['dec_LRG']
print(len(decLRG))
print(decLRG[0:10])

In [44]:
data0 = np.load('/Users/mtownsend/Documents/LRG_Data/expanded-data/new-cats/050-080/mw_trans_z_BKG/mw_transmission_z_BKG-050-080-dr9-0.npz')
data1 = np.load('/Users/mtownsend/Documents/LRG_Data/expanded-data/new-cats/050-080/mw_trans_z_BKG/mw_transmission_z_BKG-050-080-dr9-1.npz')
data2 = np.load('/Users/mtownsend/Documents/LRG_Data/expanded-data/new-cats/050-080/mw_trans_z_BKG/mw_transmission_z_BKG-050-080-dr9-2.npz')
data3 = np.load('/Users/mtownsend/Documents/LRG_Data/expanded-data/new-cats/050-080/mw_trans_z_BKG/mw_transmission_z_BKG-050-080-dr9-3.npz')
data4 = np.load('/Users/mtownsend/Documents/LRG_Data/expanded-data/new-cats/050-080/mw_trans_z_BKG/mw_transmission_z_BKG-050-080-dr9-4.npz')
data5 = np.load('/Users/mtownsend/Documents/LRG_Data/expanded-data/new-cats/050-080/mw_trans_z_BKG/mw_transmission_z_BKG-050-080-dr9-5.npz')
data6 = np.load('/Users/mtownsend/Documents/LRG_Data/expanded-data/new-cats/050-080/mw_trans_z_BKG/mw_transmission_z_BKG-050-080-dr9-6.npz')
data7 = np.load('/Users/mtownsend/Documents/LRG_Data/expanded-data/new-cats/050-080/mw_trans_z_BKG/mw_transmission_z_BKG-050-080-dr9-7.npz')
data8 = np.load('/Users/mtownsend/Documents/LRG_Data/expanded-data/new-cats/050-080/mw_trans_z_BKG/mw_transmission_z_BKG-050-080-dr9-8.npz')
data9 = np.load('/Users/mtownsend/Documents/LRG_Data/expanded-data/new-cats/050-080/mw_trans_z_BKG/mw_transmission_z_BKG-050-080-dr9-9.npz')
# data10 = np.load('/Users/mtownsend/Documents/LRG_Data/expanded-data/new-cats/050-080/raLRG/ra_LRGs-050-080-dr9-10.npz')
# data11 = np.load('/Users/mtownsend/Documents/LRG_Data/expanded-data/new-cats/050-080/raLRG/ra_LRGs-050-080-dr9-11.npz')
# data12 = np.load('/Users/mtownsend/Documents/LRG_Data/expanded-data/new-cats/050-080/raLRG/ra_LRGs-050-080-dr9-12.npz')
# data13 = np.load('/Users/mtownsend/Documents/LRG_Data/expanded-data/new-cats/050-080/raLRG/ra_LRGs-050-080-dr9-13.npz')
# data14 = np.load('/Users/mtownsend/Documents/LRG_Data/expanded-data/new-cats/050-080/raLRG/ra_LRGs-050-080-dr9-14.npz')
# data15 = np.load('/Users/mtownsend/Documents/LRG_Data/expanded-data/new-cats/050-080/raLRG/ra_LRGs-050-080-dr9-15.npz')
# data16 = np.load('/Users/mtownsend/Documents/LRG_Data/expanded-data/new-cats/050-080/raLRG/ra_LRGs-050-080-dr9-16.npz')
# data17 = np.load('/Users/mtownsend/Documents/LRG_Data/expanded-data/new-cats/050-080/raLRG/ra_LRGs-050-080-dr9-17.npz')
# data18 = np.load('/Users/mtownsend/Documents/LRG_Data/expanded-data/new-cats/050-080/raLRG/ra_LRGs-050-080-dr9-18.npz')
# data19 = np.load('/Users/mtownsend/Documents/LRG_Data/expanded-data/new-cats/050-080/raLRG/ra_LRGs-050-080-dr9-19.npz')
# data20 = np.load('/Users/mtownsend/Documents/LRG_Data/expanded-data/new-cats/050-080/raLRG/ra_LRGs-050-080-dr9-20.npz')
# data21 = np.load('/Users/mtownsend/Documents/LRG_Data/expanded-data/new-cats/050-080/raLRG/ra_LRGs-050-080-dr9-21.npz')
# data22 = np.load('/Users/mtownsend/Documents/LRG_Data/expanded-data/new-cats/020-060/mw_trans_z_BKG/mw_transmission_z_BKG-020-060-dr9-22.npz')
# data23 = np.load('/Users/mtownsend/Documents/LRG_Data/expanded-data/new-cats/020-060/mw_trans_z_BKG/mw_transmission_z_BKG-020-060-dr9-23.npz')
# data24 = np.load('/Users/mtownsend/Documents/LRG_Data/mw_transmission_z_BKG-000-060-dr9-24.npz')
# data25 = np.load('/Users/mtownsend/Documents/LRG_Data/expanded-data/new-cats/ra_LRGs-000-060-dr9-25.npz')
# data26 = np.load('/Users/mtownsend/Documents/LRG_Data/expanded-data/new-cats/ra_LRGs-000-060-dr9-26.npz')
# data27 = np.load('/Users/mtownsend/Documents/LRG_Data/expanded-data/new-cats/ra_LRGs-000-060-dr9-27.npz')

dataTOT = np.concatenate([data0['mw_transmission_z_BKG'], data1['mw_transmission_z_BKG'], data2['mw_transmission_z_BKG'], data3['mw_transmission_z_BKG'], data4['mw_transmission_z_BKG'],
                         data5['mw_transmission_z_BKG'], data6['mw_transmission_z_BKG'], data7['mw_transmission_z_BKG'], data8['mw_transmission_z_BKG'], data9['mw_transmission_z_BKG']]) #,
#                          data10['mw_transmission_z_BKG'], data11['mw_transmission_z_BKG'], data12['mw_transmission_z_BKG'], data13['mw_transmission_z_BKG'], data14['mw_transmission_z_BKG'], 
#                          data15['mw_transmission_z_BKG'], data16['mw_transmission_z_BKG'], data17['mw_transmission_z_BKG'], data18['mw_transmission_z_BKG'], data19['mw_transmission_z_BKG'],
#                          data20['mw_transmission_z_BKG'], data21['mw_transmission_z_BKG'], data22['mw_transmission_z_BKG'], data23['mw_transmission_z_BKG']])

print(len(dataTOT))

# Save the new npz file
np.savez('/Users/mtownsend/Documents/LRG_Data/expanded-data/new-cats/mw_transmission_z_BKG-050-080-TOT.npz', dataTOT)


data0.close()
data1.close()
data2.close()
data3.close()
data4.close()
data5.close()
data6.close()
data7.close()
data8.close()
data9.close()
# data10.close()
# data11.close()
# data12.close()
# data13.close()
# data14.close()
# data15.close()
# data16.close()
# data17.close()
# data18.close()
# data19.close()
# data20.close()
# data21.close()
# data22.close()
# data23.close()
# data24.close()
# data25.close()
# data26.close()
# data27.close()


18226594
